## Исследование данных

In [2]:
import json
import pandas as pd

#### Импортирование данных Мурманска

In [ ]:
a = []
for n in range(1, 50):
    content = pd.read_json(f"http://31.172.135.12/api/events?page={n}", lines=True)
    a.extend(content['data'][0])

In [ ]:
df = pd.DataFrame(a)

In [67]:
df = pd.read_csv('data/data.csv')

In [69]:
df.to_csv('data/data.csv', index=False)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      735 non-null    int64  
 1   user_id                 0 non-null      object 
 2   title                   735 non-null    object 
 3   description             735 non-null    object 
 4   images                  0 non-null      object 
 5   started_at              734 non-null    object 
 6   stopped_at              735 non-null    object 
 7   created_at              735 non-null    object 
 8   updated_at              735 non-null    object 
 9   deleted_at              0 non-null      object 
 10  annotation              722 non-null    object 
 11  city_id                 735 non-null    int64  
 12  place_id                109 non-null    float64
 13  label                   712 non-null    object 
 14  status                  735 non-null    ob

In [303]:
nan_cols = df.columns[df.isna().all(0)]
nan_cols

Index(['user_id', 'images', 'deleted_at', 'age_restriction_id', 'season_id',
       'rating', 'user_rating', 'rating_count'],
      dtype='object')

In [304]:
df.drop(nan_cols, axis=1, inplace=True)

In [306]:
df.head()

,id,title,description,started_at,stopped_at,created_at,updated_at,annotation,city_id,place_id,label,status,category_id,tourism_type_id,accessible_environment,adapter,external_id,external_data
0,56,Выставка «Эклектика города»,<p>В экспозиции выставки будут представлены пе...,2020-12-11 11:00:00,2021-01-10 19:00:00,2020-11-19 18:16:17,2020-12-02 13:29:44,Представлены пейзажи города в исполнении мурма...,12,109.0,От 30 ₽,active,4,NaN,False,culture-ru,819726.0,"{""_id"":819726,""name"":""Выставка «Эклектика горо..."
1,233,"Конкурс «Фантазируем, мечтаем, книги сами сочи...",<p>МБУК «ЦБС ЗАТО Александровск» объявляет XXV...,2020-12-10 10:00:00,2021-01-29 18:00:00,2020-12-15 16:31:10,2020-12-23 02:19:09,Участники получат возможность проявить свои ли...,13,NaN,Бесплатно,active,2,NaN,False,culture-ru,853933.0,"{""_id"":853933,""name"":""Конкурс «Фантазируем, ме..."
2,25,"«Здравствуй, солнце!».","<p>«Здравствуй, Солнце!» — праздник в честь ок...",2021-01-01 00:00:00,2021-01-31 00:00:00,2020-09-29 12:21:50,2021-01-29 16:37:49,NaN,12,39.0,БЕСПЛАТНО,active,1,2.0,False,NaN,NaN,NaN
3,23,Международный кинофестиваль «Северный характер»,"<p>Социально-культурный проект, в ходе которог...",2021-03-01 00:00:00,2021-04-30 00:00:00,2020-09-29 12:15:13,2020-10-28 15:43:19,NaN,12,NaN,СКОРО,active,1,4.0,False,NaN,NaN,NaN
4,27,День рыбака,<p>День рыбака в Мурманске всегда отмечается с...,2021-07-01 00:00:00,2021-07-31 00:00:00,2020-09-29 12:35:17,2020-10-01 14:36:30,NaN,12,39.0,бесплатно,active,7,4.0,False,NaN,NaN,NaN


In [45]:
df.nunique()

id                        735
title                     714
description               734
started_at                466
stopped_at                483
created_at                701
updated_at                735
annotation                680
city_id                    12
place_id                   22
label                      28
status                      1
category_id                 9
tourism_type_id             4
accessible_environment      2
adapter                     1
external_id               721
external_data             721
dtype: int64

#### Данные с kudago по Москве

In [12]:
import json
resp1 = json.load(open('data/response_1616889786114.json', encoding='utf-8'))

In [13]:
df = pd.DataFrame(([list(*p.items()) for p in resp1]), columns=['Description', 'spam'])

In [14]:
df.drop_duplicates(inplace=True)

In [292]:
!pip install html2text

In [15]:
import html2text

#### Предобработка

In [16]:
df.Description = df.Description.apply(html2text.html2text)

def normalize(df: pd.Series):
    df = df.str.lower()
    df = df.str.replace('[\W_]', ' ', regex=True)
    df = df.apply(lambda x: [i for i in x.split() if len(i) > 1])
    return df

df.Description = normalize(df.Description)

#### Обучение Word2Vec и усреднение векторов слов

In [19]:
import numpy as np
import gensim
from gensim.models import Word2Vec

def word_averaging(wv, words):
    mean = np.zeros((wv.vector_size,))
    
    for word in words:
        if word in wv.vocab:
            mean += wv.get_vector(word)

    mean = gensim.matutils.unitvec(mean)
    return mean

def word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list])

In [20]:
model = Word2Vec(df.Description, size=160, window=30, min_count=2, workers=12, iter=10, sg=1)

## Предсказания для мурманских мероприятий

In [21]:
df1 = pd.read_csv('data/data.csv')

In [22]:
df1.description = df1.description.apply(html2text.html2text)

df1['normalized'] = normalize(df1.description)
X_wv = word_averaging_list(model.wv, df1.normalized)

#### Обучение ближайших соседей

In [23]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_jobs=12)
nn.fit(X_wv)

NearestNeighbors(n_jobs=12)

### Первый пример

#### Описание мероприятия

In [65]:
n_sample = 0
print(df1.loc[n_sample].description[:500])

В экспозиции выставки будут представлены пейзажи города в исполнении
мурманских художников из частной коллекции Алексея Макарова﻿

Алексей Макаров с 1997 по настоящее время ведущий актер драматического театра
Северного флота.

На выставке зрители увидят Портреты Мурманска в исполнении разных художников:
Виктории Зубицкой, Татьяны Ковалёвой, Николая Ковалёва, Николая Морозова,
Виталия Бубенцова, Анатолия Шаковца, Анастасии Берёзы, Никиты Ковалёва и
других известных художников Мурманска и Мурманск


#### Похожие на данное мероприятие

In [42]:
dist, nums = nn.kneighbors(X_wv[None, n_sample], 6)
print(*df1.iloc[nums[0, 1:]].title.str[:139], sep='\n_______\n')

Выставка «Отечественное изобразительное искусство XVIII-XX веков»
_______
Выставка «Саамские краски»
_______
Выставка «Любительское изобразительное и ДПИ Мурманской области»
_______
Выставка «Александр III и Александровск»
_______
Виртуальная выставка «Мир изобразительного искусства»


### Второй пример

#### Описание мероприятия

In [63]:
n_sample = 15
print(df1.loc[n_sample].description[:495])

Ежегодно в середине лета на берегу океана принимает гостей арктический
фестиваль «Териберка». Известные российские музыканты, живые выступления и
незабываемая атмосфера праздника притягивают сюда тысячи туристов.

Фишка фестиваля — открытые мастер-классы именитых шеф-поваров. Лучшие из
лучших здесь соревнуются в мастерстве и угощают гостей блюдами арктической
кухни. Морские деликатесы, свежайшие морепродукты, северные ягоды и необычное
мясо с ресторанной подачей — настоящий рай для гурманов


#### Похожие на данное мероприятие

In [45]:
dist, nums = nn.kneighbors(X_wv[None, n_sample], 6)
print(*df1.iloc[nums[0, 1:]].title.str[:139], sep='\n_______\n')

Традиционные соревнования  Поморская гребная регата
_______
Праздник севера
_______
«Фольклор Град»
_______
Международный культурный фестиваль «Птица Баренц»
_______
Концерт «Зимняя жара»


### Третий пример

#### Описание мероприятия

In [57]:
n_sample = 25
print(df1.loc[n_sample].description[:540])

Одна из самых героических и романтических морских профессий – моряк-подводник.
Об отличающей их отваге, полной самоотдаче делу, железной дисциплине и морском
товариществе сложено немало песен, с которыми библиотекари предлагают
познакомиться.

В ходе встречи прозвучит стихотворение И. Фисановича «Строевая подводная»,
ставшее неофициальной песней–гимном подводников в годы Великой Отечественной
войны.

Присутствующим будет рассказано и об истории создания известной музыкальной
композиции «Усталая подлодка» А. Пахмутовой и Н. Добронравов


#### Похожие на данное мероприятие

In [48]:
dist, nums = nn.kneighbors(X_wv[None, n_sample], 6)
print(*df1.iloc[nums[0, 1:]].title.str[:139], sep='\n_______\n')

Колесо истории «Память в сердце храня»
_______
Презентация книги «Оленетранспортные батальоны в годы Великой Отечественной войны»
_______
Час мужества «Свет Сталинградской победы»
_______
Музыкальная гостиная «Образ пленительный, образ прекрасный»
_______
Историческое досье «Люди мира на минуту встаньте!»
